# Wind Speed Forecast in DK1

Fetch wind speed (10m) forecast data for DK1 (Western Denmark) using Open-Meteo.

In [4]:
# 1. Imports

import requests
import pandas as pd

from datetime import datetime, timedelta

In [6]:
# 2. fetch_historical_weather_ecmwf.py

# Set location
latitude = 56.1629     # Aarhus
longitude = 10.2039
days_back = 720        # Useless
start_str = "2016-01-01"
end_str = "2016-12-31"

url = (
    "https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    "&hourly=windspeed_10m,winddirection_10m,temperature_2m,surface_pressure,relativehumidity_2m"
    f"&start_date={start_str}&end_date={end_str}"
    "&timezone=auto"
)

print(f"Fetching 2016 weather data for Aarhus...")
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_weather = pd.DataFrame({
    "timestamp": data["hourly"]["time"],
    "windspeed_10m": data["hourly"]["windspeed_10m"],
    "winddirection_10m": data["hourly"]["winddirection_10m"],
    "temperature_2m": data["hourly"]["temperature_2m"],
    "surface_pressure": data["hourly"]["surface_pressure"],
    "relativehumidity_2m": data["hourly"]["relativehumidity_2m"],
})

df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"])
df_weather = df_weather.sort_values("timestamp")

# Save
output_path = "../data/weather_2016_ecmwf.csv"
df_weather.to_csv(output_path, index=False)

print("✅ Saved 2016 historical weather data to:", output_path)
print(df_weather.head())


Fetching 2016 weather data for Aarhus...
✅ Saved 2016 historical weather data to: ../data/weather_2016_ecmwf.csv
            timestamp  windspeed_10m  winddirection_10m  temperature_2m  \
0 2016-01-01 00:00:00           13.7                194             4.7   
1 2016-01-01 01:00:00           12.4                188             4.4   
2 2016-01-01 02:00:00           12.2                182             4.0   
3 2016-01-01 03:00:00           13.0                186             3.6   
4 2016-01-01 04:00:00           13.6                192             3.5   

   surface_pressure  relativehumidity_2m  
0            1015.8                   97  
1            1016.1                   97  
2            1016.6                   97  
3            1017.0                   97  
4            1017.3                   97  


In [8]:
# Merge training data

df_weather = pd.read_csv("../data/weather_2016_ecmwf.csv", parse_dates=["timestamp"])

df_power = pd.read_csv("../data/wind_power.csv", parse_dates=["utc_timestamp"])
df_power.rename(columns={"utc_timestamp": "timestamp"}, inplace=True)

df_power["timestamp"] = df_power["timestamp"].dt.tz_localize(None)

df = pd.merge(df_power, df_weather, on="timestamp", how="inner")

df.to_csv("../data/merged_training_data_2016.csv", index=False)
print("✅ 合并完成，已保存至 ../data/merged_training_data_2016.csv")
print(df.head())

✅ 合并完成，已保存至 ../data/merged_training_data_2016.csv
            timestamp  DK_1_wind_onshore_generation_actual  windspeed_10m  \
0 2016-01-01 00:00:00                               830.31           13.7   
1 2016-01-01 01:00:00                               701.10           12.4   
2 2016-01-01 02:00:00                               696.06           12.2   
3 2016-01-01 03:00:00                               699.83           13.0   
4 2016-01-01 04:00:00                               747.99           13.6   

   winddirection_10m  temperature_2m  surface_pressure  relativehumidity_2m  
0                194             4.7            1015.8                   97  
1                188             4.4            1016.1                   97  
2                182             4.0            1016.6                   97  
3                186             3.6            1017.0                   97  
4                192             3.5            1017.3                   97  


In [3]:
# 2. Read forecasts data from ECMWF and save it

# Aarhus, DK1 region
latitude = 56.1629
longitude = 10.2039

url = (
    "https://api.open-meteo.com/v1/forecast?"
    f"latitude={latitude}&longitude={longitude}"
    "&hourly=windspeed_10m,winddirection_10m,temperature_2m,surface_pressure,relativehumidity_2m"
    "&forecast_days=3"
    "&timezone=auto"
)

response = requests.get(url)
data = response.json()

df_forecast = pd.DataFrame({
    "timestamp": data["hourly"]["time"],
    "windspeed_10m": data["hourly"]["windspeed_10m"],
    "winddirection_10m": data["hourly"]["winddirection_10m"],
    "temperature_2m": data["hourly"]["temperature_2m"],
    "surface_pressure": data["hourly"]["surface_pressure"],
    "relativehumidity_2m": data["hourly"]["relativehumidity_2m"],
})

df_forecast["timestamp"] = pd.to_datetime(df_forecast["timestamp"])

print(df_forecast.head())
df_forecast.to_csv("../data/forecast_wind_dk1.csv", index=False)

            timestamp  windspeed_10m  winddirection_10m  temperature_2m  \
0 2025-04-06 00:00:00           10.8                  8             5.0   
1 2025-04-06 01:00:00           11.5                 14             4.8   
2 2025-04-06 02:00:00           10.4                  7             4.4   
3 2025-04-06 03:00:00            9.7                  6             4.1   
4 2025-04-06 04:00:00            9.0                349             3.6   

   surface_pressure  relativehumidity_2m  
0            1024.9                   63  
1            1025.1                   66  
2            1025.6                   65  
3            1025.6                   66  
4            1026.0                   66  
